In [38]:
import pandas as pd
import numpy as np

In [39]:
people = pd.read_csv('/content/202402_연령별인구현황_5단위.csv', encoding = 'cp949')
basic = pd.read_csv('/content/강서구_기초생활수급자 동별 현황_20200531.csv')
mov_2018 = pd.read_csv('/content/강서구_유동인구_2018.csv')
mov_2019 = pd.read_csv('/content/강서구_유동인구_2019.csv')
one_man = pd.read_csv('/content/서울특별시 강서구_행정동별 1인가구 현황_2018.csv', encoding = 'cp949')

# 데이터 전처리

## 인구 현황 전처리

In [40]:
# '행정구역' 컬럼에서 "서울특별시 서울특별시 강서구" 문자열을 제거합니다.
df_1 = people.copy()

# '행정구역' 컬럼에서 "서울특별시 서울특별시 강서구" 문자열을 제거합니다.
df_1['행정구역'] = df_1['행정구역'].str.replace("서울특별시 강서구", "")

# 컬럼 이름에서 "2024년02월_" 부분을 제거합니다.
df_1.columns = [col.replace("2024년02월_", "") for col in df_1.columns]

df_1.head()

,행정구역,거주자_총인구수,거주자_연령구간인구수,거주자_0~4세,거주자_5~9세,거주자_10~14세,거주자_15~19세,거주자_20~24세,거주자_25~29세,거주자_30~34세,...,여_거주자_55~59세,여_거주자_60~64세,여_거주자_65~69세,여_거주자_70~74세,여_거주자_75~79세,여_거주자_80~84세,여_거주자_85~89세,여_거주자_90~94세,여_거주자_95~99세,여_거주자_100세 이상
0,서울특별시,"558,886","558,886","13,150","17,485","20,343","21,196","29,199","49,218","53,295",...,"20,474","22,602","20,618","13,698","9,819","7,022","4,011","1,628",444,65
1,염창동,"41,098","41,098","1,457","1,944","2,264","2,083","2,165","3,088","3,717",...,"1,358","1,325","1,014",662,400,324,161,64,16,1
2,등촌제1동,"24,836","24,836",689,551,507,616,"1,352","3,317","3,869",...,762,740,581,376,295,260,210,93,28,4
3,등촌제2동,"19,165","19,165",413,528,654,733,"1,041","1,642","1,842",...,812,872,736,508,296,213,124,58,14,2
4,등촌제3동,"29,182","29,182",647,756,732,878,"1,344","1,871","2,465",...,"1,273","1,521","1,469",898,683,597,444,180,51,7


### 연령별 인구 현황 총합, 남, 여 분리

In [41]:
df_total = df_1.iloc[:,:24]
df_man = df_1.iloc[:,24:47]
df_woman = df_1.iloc[:,47:70]

In [42]:
# '행정 구역' 컬럼 추출
행정구역 = df_1.iloc[:, 0]

# 앞서 생성한 데이터 프레임에 '행정 구역' 컬럼을 앞에 추가
df_man.insert(0, '행정구역', 행정구역)
df_woman.insert(0, '행정구역', 행정구역)

In [43]:
df_man.at[0, '행정구역'] = '서울특별시 강서구'
df_woman.at[0, '행정구역'] = '서울특별시 강서구'
df_total.at[0, '행정구역'] = '서울특별시 강서구'

In [44]:
# 각 데이터 프레임을 CSV 파일로 저장합니다. 인코딩은 한국어 데이터를 위해 'utf-8-sig'를 사용합니다.
df_total.to_csv('df_total.csv', index=False, encoding='utf-8-sig')
df_man.to_csv('df_man.csv', index=False, encoding='utf-8-sig')
df_woman.to_csv('df_woman.csv', index=False, encoding='utf-8-sig')

### 데이터 병합용 인구 현황 데이터 생성

In [45]:
import pandas as pd

# df_total에서 앞의 3개 컬럼 선택
df_total_selected = df_total.iloc[:, :3]

# df_woman에서 2, 3번째 컬럼 선택 (파이썬에서 인덱스는 0부터 시작하므로 1, 2를 사용)
df_woman_selected = df_woman.iloc[:, [1, 2]]

# df_man에서 2, 3번째 컬럼 선택
df_man_selected = df_man.iloc[:, [1, 2]]

# 세 데이터 프레임을 가로로 결합
df_concat = pd.concat([df_total_selected, df_woman_selected, df_man_selected], axis=1)

# 결과 확인
df_concat.to_csv('df_concat.csv', index=False, encoding='utf-8-sig')

In [78]:
df_concat.rename(columns={df_concat.columns[0]: '행정동'}, inplace=True)
df_concat = df_concat.drop(columns = ['거주자_연령구간인구수', '여_거주자_연령구간인구수', '남_거주자_연령구간인구수'])
df_concat.head(1)

,행정동,거주자_총인구수,여_거주자_총인구수,남_거주자_총인구수
0,서울특별시 강서구,"558,886","291,260","267,626"


## 기초수급자 데이터 행정동 전처리

In [47]:
# 행정동 통일을 위해 unique 뽑아 보기

unique_values = df_concat['행정동'].unique()
print(unique_values)

['서울특별시 강서구' ' 염창동' ' 등촌제1동' ' 등촌제2동' ' 등촌제3동' ' 화곡제1동' ' 화곡제2동' ' 화곡제3동'
 ' 화곡제4동' ' 화곡본동' ' 화곡제6동' ' 화곡제8동' ' 가양제1동' ' 가양제2동' ' 가양제3동' ' 발산제1동'
 ' 우장산동' ' 공항동' ' 방화제1동' ' 방화제2동' ' 방화제3동']


In [63]:
basic.head()

,읍면동,합계_가구수,합계_수급권자수,일반수급자_가구수,일반수급자_수급권자수,시설수급자 수
0,NaN,가구수,수급권자수,가구수,수급권자수,수급권자수
1,서울특별시 강서구,"18,219","25,187","18,075","25,043",144
2,염창동,129,179,129,179,0
3,등촌1동,163,227,163,227,0
4,등촌2동,238,357,225,344,13


In [64]:
print(basic['읍면동'].unique())

[nan '서울특별시 강서구' '염창동' '등촌1동' '등촌2동' '등촌3동' '화곡1동' '화곡2동' '화곡3동' '화곡4동'
 '화곡본동' '화곡6동' '화곡8동' '가양1동' '가양2동' '가양3동' '발산1동' '우장산동' '공항동' '방화1동'
 '방화2동' '방화3동']


In [67]:
# 필요 없는 column 삭제 및 이름 재지정

basic.columns = ['읍면동', '합계_가구수', '합계_수급권자수', '일반수급자_가구수', '일반수급자_수급권자수', '시설수급자 수']
basic = basic.drop(basic.index[0])
basic.at[1, '읍면동'] = '서울특별시 강서구'

In [69]:
# 기초수급자 데이터 행정동 전처리

basic.rename(columns={basic.columns[0]: '행정동'}, inplace=True)

def add_je(name):
    import re
    # 정규 표현식을 사용하여 '숫자동' 패턴을 찾고, '제숫자동'으로 변경
    new_name = re.sub(r'(\d)동', r'제\1동', name)
    return new_name

basic['행정동'] = basic['행정동'].apply(add_je)

## 1인 가구 전처리

In [72]:
# 두 번째 열(B)의 모든 값들을 더함
sum_column = one_man.iloc[:, 1].sum()  # 'iloc[:, 1]'로 두 번째 열을 선택

# 새로운 데이터 (여기서는 두 번째 열의 합)를 첫 번째 행에 추가하기 위해
# 새로운 데이터 프레임을 생성하고 원본 데이터 프레임과 결합
new_row = pd.DataFrame({one_man.columns[1]: sum_column}, index=[0])  # 새로운 행 데이터 프레임 생성

# 새로운 행을 첫 번째 위치에 추가하기 위해 concat 사용
one_updated = pd.concat([new_row, one_man], ignore_index=True)

In [73]:
one_updated .at[0, '행정동'] = '서울특별시 강서구'

In [75]:
one_updated.drop(columns = ['데이터기준일자'], inplace=True)

## 데이터 병합

- 연령별 인구 현황 : df_concat
- 기초수급자 현황 : basic
- 1인 가구 현황 : one_updated

In [80]:
print("1인 가구 현황 데이터 : ",one_updated .shape)
print("기초 수급자 현황 데이터 : ",basic.shape)
print("인구 현황 데이터 : ",df_concat.shape)

1인 가구 현황 데이터 :  (21, 2)
기초 수급자 현황 데이터 :  (21, 6)
인구 현황 데이터 :  (21, 4)


In [81]:
# 1인 가구와 기초 수급자 데이터 프레임을 '행정동'을 기준으로 합치기
merge_1 = pd.merge(one_updated , basic, on='행정동', how='inner')

merge_1.shape

(21, 7)

In [97]:
# df_concat의 '행정동' 열에서 앞뒤 공백 제거
df_concat['행정동'] = df_concat['행정동'].str.strip()

# merge_1의 '행정동' 열에서 앞뒤 공백 제거 (필요한 경우)
merge_1['행정동'] = merge_1['행정동'].str.strip()

# 이제 두 데이터 프레임 병합 시도
df_merge = pd.merge(df_concat, merge_1, on='행정동', how='inner')

# 병합된 데이터 프레임의 크기 확인
print(df_merge.shape)

(21, 10)


In [99]:
print("병합 데이터 널 데이터 수 : ", df_merge.isnull().sum().sum())

병합 데이터 널 데이터 수 :  0


In [100]:
df_merge.to_csv('df_merge.csv', index=False, encoding='utf-8-sig')